<H1>Python submissions to MyTargets</H1>

One of the additional features on ExoFOP is the <A HREF=https://exofop.ipac.caltech.edu/tess/mytargets.php>'MyTargets'</A> functionality that lets users request information on if user-selected TOIs have received updates on ExoFOP by submitting a file and receiving an email. This list can be submitted manually through ExoFOP, and the ExoFOP help page explains how to submit this as a curl request that can be set up as a scheduled cron job.

Here we demonstrate how MyTargets requests can be scheduled using Python by first creating a target list using tools in the first notebook. Using `pandas` we first load the TOI data from ExoFOP as we did in Notebook 01.

In [ ]:
import pandas as pd
TOI_url="https://exofop.ipac.caltech.edu/tess/download_toi.php?sort=toi&output=pipe"
TOI_df=pd.read_csv(TOI_url, delimiter='|', index_col=1)
print(TOI_df)

To generate a list of targets for our MyTargets submission, we will pick planet candidates that are warm Jupiters as the subset of TOIs that we would like to get information on. To do that, are going to use `pandas` .loc indexing option and some boolean statements to select just a subset of the TOIs.  
We start by only selecting TOIs that have periods between 10 and 20 days.  
Next we only select planets that are larger than 8 Earth radii.  
Finally, we exclude any TOIs that have had their disposition set as 'CP' (confirmed planet), 'FP' (false positive), or 'KP' (known planet).

The MyTargets submission file format contains three columns; the TIC ID, a code for the 'data' that we would like, and and a number for the 'days' of data we would like.

The options for the 'data' parameter are as follows:  
A	all  
F	files  
OI	imaging observations  
OS	spectroscopy observations  
OT	time series observations  

The 'days' are how far back we would like our search window for new updates to go, with an integer between 1 and 30.

We add two columns to our dataframe so that for all of our TOIs, we will be getting all data updates ('data' = A) in the last 3 days (days = 3). If we wanted to, we could use a function to determine which kind of data we want, or how many days of data we want, so that these values are not uniform.

Finally, we use to_csv() to write our TOIs to a file called 'mytargets.txt'. To make it match the format that ExoFOP will be able to read in, we do not include an index column or a header row, we specify that the delimiter is '|', and we list that the only columns we want to output are 'TIC ID', 'data', and 'days', in that order.

In [ ]:
TOI_df=TOI_df.loc[(TOI_df['Period (days)']>10) & (TOI_df['Period (days)']<20)]
TOI_df=TOI_df.loc[(TOI_df['Planet Radius (R_Earth)']>8)]
TOI_df=TOI_df.loc[(TOI_df['TFOPWG Disposition'] != 'CP') & (TOI_df['TFOPWG Disposition'] != 'FP') & (TOI_df['TFOPWG Disposition'] != 'KP')]
TOI_df['data']='A'
TOI_df['days']=3
TOI_df.to_csv('mytargets.txt', index=False, columns=['TIC ID','data','days'], sep='|', header=False)

Now that we have the MyTargets file, we import the subprocess package as well as functions from the ExoFOP and my_targets modules in this project.create_cookie shell script. We then submit our MyTargets.txt file using the my_targets function.

First we use ExoFOP_parameters to load all the necessary user-based environmental parameters by reading in the file that we created in notebook 00 into a Python dictionary called 'params'.

Our next step is to run the create_cookie shell script, which will use that same file and generate a user-specific cookie that will be needed to submit 'MyTarget' requests to ExoFOP. We also specify the submission_url that corresponds to the page for uploading a 'MyTarget' request.

Finally, we use the my_targets function to submit our list of TOIs. This function requires our formatted TOI file (here we have named it 'mytargets.txt'), the file name of our cookie (this name is generated based off of username, and is stored in the 'params' dictionary), and the submission_url for 'MyTargets'.

In [ ]:
from ExoFOP import ExoFOP_parameters
import subprocess
from my_targets import my_targets

params=ExoFOP_parameters('ExoFOP_info.txt')
subprocess.call(['./create_cookie.sh','ExoFOP_info.txt'])
submission_url='https://exofop.ipac.caltech.edu/tess/mytargets_upload.php'


my_targets('mytargets.txt', params['cookie'], submission_url)

If the above has run successfully, a message should appear saying "Your file was uploaded successfully and you will receive email notification about your targets" followed shortly by receiving an email from exofop-support@ipac.caltech.edu to the email address associated with your ExoFOP account.